<a href="https://colab.research.google.com/github/pvpogorelova/metrics_24_25/blob/main/sem_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Модели цензуированных данных.**

In [1]:
!pip install py4etrics # tobit, heckit
!pip install wooldridge # datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 21.6 MB/s eta 0:00:00


In [2]:
from numpy import log, exp, cos, sin, sqrt, max, min
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats import norm
import patsy

from py4etrics.truncreg import Truncreg
from py4etrics.tobit import Tobit
from py4etrics.heckit import Heckit
from py4etrics.hetero_test import het_test_probit

import wooldridge

Датасет **htv** из пакета **wooldridge** в Python содержит данные о заработной плате и образовании, используемые в эконометрическом анализе. Этот датасет часто применяется для изучения взаимосвязи между уровнем образования, профессиональной подготовкой и доходами.

**Описание переменных**

Датасет включает следующие основные переменные:

* wage – почасовая заработная плата.

* abil – показатель способностей (например, результаты тестов).

* educ – количество лет образования.

* exper – опыт работы (в годах).

* tenure – стаж на текущем месте работы (в годах).

* male – бинарная переменная (1 = мужчина, 0 = женщина).

* married – бинарная переменная (1 = женат/замужем, 0 = нет).

* south – бинарная переменная (1 = проживание на юге США, 0 = нет).

* urban – бинарная переменная (1 = проживание в городе, 0 = нет).

In [3]:
htv = wooldridge.data('htv')
wooldridge.data('htv', description=True)

name of dataset: htv
no of variables: 23
no of observations: 1230

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| wage     | hourly wage, 1991               |
| abil     | abil. measure, not standardized |
| educ     | highest grade completed by 1991 |
| ne       | =1 if in northeast, 1991        |
| nc       | =1 if in nrthcntrl, 1991        |
| west     | =1 if in west, 1991             |
| south    | =1 if in south, 1991            |
| exper    | potential experience            |
| motheduc | highest grade, mother           |
| fatheduc | highest grade, father           |
| brkhme14 | =1 if broken home, age 14       |
| sibs     | number of siblings              |
| urban    | =1 if in urban area, 1991       |
| ne18     | =1 if in NE, age 18             |
| nc18     | =1 if in NC, age 18             |
| south18  | =1 if in south, age 18          |
| west18   | =1 if in west, age 18      

In [4]:
htv.head(5)

,wage,abil,educ,ne,nc,west,south,exper,motheduc,fatheduc,...,ne18,nc18,south18,west18,urban18,tuit17,tuit18,lwage,expersq,ctuit
0,12.019231,5.027738,15,0,0,1,0,9,12,12,...,1,0,0,0,1,7.582914,7.260242,2.486508,81,-0.322671
1,8.912656,2.037170,13,1,0,0,0,8,12,10,...,1,0,0,0,1,8.595144,9.499537,2.187472,64,0.904392
2,15.514334,2.475895,15,1,0,0,0,11,12,16,...,1,0,0,0,1,7.311346,7.311346,2.741764,121,0.000000
3,13.333333,3.609240,15,1,0,0,0,6,12,12,...,1,0,0,0,1,9.499537,10.162070,2.590267,36,0.662534
4,11.070110,2.636546,13,1,0,0,0,15,12,15,...,1,0,0,0,1,7.311346,7.311346,2.404249,225,0.000000


In [6]:
# Для начала оценим с помощью МНК обычную модель регрессии для логарифма зарплаты
formula= 'lwage ~ 1 + educ + abil + exper + nc + west + south + urban'

res_ols = smf.ols(formula, data=htv).fit(cov_type='HC3') # всегда используем робастные стандартные ошибки
print(res_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     43.89
Date:                Sun, 20 Apr 2025   Prob (F-statistic):           1.57e-55
Time:                        16:46:09   Log-Likelihood:                -939.84
No. Observations:                1230   AIC:                             1896.
Df Residuals:                    1222   BIC:                             1937.
Df Model:                           7                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3987      0.194      2.057      0.0

# **Урезанные данные.**

Теперь предположим, что при сборе данных в выбору входили только те, чьи доходы не выше max(htv['lwage']) В этом случае мы имеем дело с урезанной выборкой.

In [7]:
max(htv['lwage'])

4.514251708984375

Как известно, если данные урезанные, то МНК оценки будут смещенными. И по идее должны отличаться от ММП оценок, которые в этом случае будут несмещенными. Оценим модель для урезанных данных с помощью ММП.

In [9]:
formula = 'lwage ~ 1 + educ + abil + exper + nc + west + south + urban'

res_trunc = Truncreg.from_formula(formula, data=htv, right=max(htv['lwage'])).fit()
print(res_trunc.summary())

# Сравните полученные результаты с результатами оценивания модели выше (МНК)

Optimization terminated successfully.
         Current function value: 0.763912
         Iterations: 257
         Function evaluations: 406
                         Truncreg Regression Results                          
Dep. Variable:                  lwage   Pseudo R-squ:                    0.148
Model:                       Truncreg   Log-Likelihood:                 -939.6
Method:            Maximum Likelihood   LL-Null:                       -1103.3
Date:                Sun, 20 Apr 2025   LL-Ratio:                        327.3
Time:                        16:47:53   LLR p-value:                     0.000
No. Observations:                1230   AIC:                            1895.2
Df Residuals:                    1222   BIC:                            1936.1
Df Model:                           7   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

In [11]:
# Тестирование гипотез
# Проверить гпиотезу о параметрах можно с помощью одного ихз следующих тестов: LR,LM,Wald.
# Для примера проверим гипотезу вида "educ+abil=exper" с помощью теста Вальда
print(res_trunc.wald_test('educ+abil=exper').summary())

<Wald test (chi2): statistic=[[233.21071339]], p-value=1.188940582819191e-52, df_denom=1>


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1912: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


Так как p-value = 0, то гипотеза $H_1$ не отвергается, то есть сумма коэффиицентов при $educ$ и $abil$ не равна коэффициенту при $exper$ на любом разумном уровне значимости.

In [12]:
# Прогнозные значения зависимой переменной и остатки модели
y_hat = res_trunc.fittedvalues
u_hat = res_trunc.resid

# **Модели цензурированных данных.**

**Tobit модель**

Датасет **mroz** из пакета **wooldridge** содержит данные о занятости и заработной плате женщин, собранные в рамках исследования рынка труда. Этот датасет часто используется в эконометрике для анализа факторов, влияющих на участие женщин в рабочей силе, а также для изучения гендерных различий в оплате труда.

**Описание переменных**

Основные переменные в датасете:

* inlf (in labor force) – бинарная переменная (1 = женщина работает, 0 = не работает).

* hours – количество отработанных часов в год.

* kidslt6 – число детей младше 6 лет.

* kidsge6 – число детей от 6 до 18 лет.

* age – возраст женщины.

* educ – количество лет образования.

* wage – почасовая заработная плата (NA для неработающих).

* repwage – предполагаемая заработная плата (на основе опроса).

* hushrs – количество рабочих часов мужа в год.

* husage – возраст мужа.

* huseduc – образование мужа (годы).

* huswage – почасовая зарплата мужа.

* faminc – общий семейный доход.

* mtr – предельная налоговая ставка.

* exper – трудовой стаж (в годах).

* expersq – квадрат стажа (exper^2).

In [13]:
mroz = wooldridge.data('mroz')
wooldridge.data('mroz', description=True)

name of dataset: mroz
no of variables: 22
no of observations: 753

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| inlf     | =1 if in lab frce, 1975         |
| hours    | hours worked, 1975              |
| kidslt6  | # kids < 6 years                |
| kidsge6  | # kids 6-18                     |
| age      | woman's age in yrs              |
| educ     | years of schooling              |
| wage     | est. wage from earn, hrs        |
| repwage  | rep. wage at interview in 1976  |
| hushrs   | hours worked by husband, 1975   |
| husage   | husband's age                   |
| huseduc  | husband's years of schooling    |
| huswage  | husband's hourly wage, 1975     |
| faminc   | family income, 1975             |
| mtr      | fed. marg. tax rte facing woman |
| motheduc | mother's years of schooling     |
| fatheduc | father's years of schooling     |
| unem     | unem. rate in county of res

In [14]:
# Создадим переменную, отвечающую за то, является ли наблюдение цензурированным
# закодируем так: 0, если нецензурированное наблюдения; -1 (если цензурированное)
# codes: 0 for non-censored part, -1 for censored part
mroz['censor'] = 0
mroz.loc[mroz['hours'] == 0, 'censor'] = -1

In [15]:
# Оценим tobit-модель
formula = 'hours ~ 1 + nwifeinc + educ + exper + expersq + age + kidslt6 + kidsge6'

res_tobit = Tobit.from_formula(formula,
                               cens=mroz['censor'],
                               left=0,
                               data=mroz).fit()

print(res_tobit.summary())

Optimization terminated successfully.
         Current function value: 5.071839
         Iterations: 2472
         Function evaluations: 3579
                              Tobit Regression Results                             
Dep. Variable:                       hours   Pseudo R-squ:                    0.034
Method:                 Maximum Likelihood   Log-Likelihood:                -3819.1
No. Observations:                      753   LL-Null:                       -3954.9
No. Uncensored Obs:                    428   LL-Ratio:                        271.6
No. Left-censored Obs:                 325   LLR p-value:                     0.000
No. Right-censored Obs:                  0   AIC:                            7654.2
Df Residuals:                          745   BIC:                            7691.2
Df Model:                                7   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------

In [16]:
y_star_hat = res_tobit.fittedvalues # y* = X \hat \beta
u_hat = res_tobit.resid # y - y*

### **Модель Хекмана**

In [17]:
# Реализуем двухшаговую процедуру оценивания модели Хекмана
# Шаг 1: Probit
# Шаг 2: OLS on regressors + Inverse Mills Ratios
y = mroz['lwage']
# регрессия первого шага
X1 = patsy.dmatrix('1 + educ + exper + expersq + nwifeinc + age + kidslt6 + kidsge6', mroz) # набор регрессоров, которые могут влиять на вероятность того, что женщина будет работать
# регрессия второго шага
X2 = patsy.dmatrix('1 + educ + exper + expersq', mroz) # набор регрессоров, которые могут влять на размер зарплаты

In [18]:
# Марица X2 должна идти перед X1
res_heckit = Heckit(y, X2, X1).fit(cov_type_2 = 'HC3')
print(res_heckit.summary())

                           Heckit Regression Results                            
Dep. Variable:                    lwage   R-squared:                       0.156
Model:                           Heckit   Adj. R-squared:                  0.150
Method:                Heckman Two-Step   F-statistics:                   26.148
Date:                  Sun, 20 Apr 2025   Prob (F-statistic):              0.000
Time:                          16:53:37   Cov in 1st Stage:            nonrobust
No. Total Obs.:                     753   Cov in 2nd Stage:                  HC3
No. Censored Obs.:                  325                                         
No. Uncensored Obs.:                428                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5781      0.305     -1.895      0.058      -1.176       0.020
x1             0.1091      0.016  

Как можно заметить из таблицы выше, коэффициент перед IMR (Inverse Mill's Ratio, он же $\lambda$ Хекмана) оказался статистически незначимым. Это означает, что эффект самоотбора отсутствует, можно оценивать модель для вероятности участия и интенсивности участия отдельно друг от друга.

В модели Хекмана коэффициент перед $\lambda$ - это ковариация ($\sigma_{ɛu}$) между ошибками двух уравнений: участия и интенсивности. И $\sigma_{ɛu} = \rho*sigma$, где $sigma$ - это стандартное отклнение ошибки в уравнении интенсивности, то есть $sigma = \sigma_{ɛ}$.

In [19]:
# Можем проверить гипотезу H_0: exper=expersq=0, kidslt6=kidsge6 с помощью теста Вальда
print(res_tobit.wald_test('exper=expersq=0, kidslt6=kidsge6').summary())

<Wald test (chi2): statistic=[[199.25009919]], p-value=6.1262329323939026e-43, df_denom=3>


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1912: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
